# Visual 3: Income and Wage Distribution Comparison (Austin TX and US)

This visualization highlights income inequality and the disconnect between rising home values and stagnant wages. It compares the distribution of household incomes in Austin with national averages, alongside a wage distribution comparison for Texas versus the U.S.

In [24]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [25]:
# Load the datasets
income_df = pd.read_csv('data files/austin_household_income.csv')
wage_df = pd.read_csv('data files/texas_wage_distribution.csv')

# Preview the data
print("Income Data Shape:", income_df.shape)
print("\nIncome Data Columns:", income_df.columns.tolist())
print("\nWage Data Shape:", wage_df.shape)
print("\nWage Data Columns:", wage_df.columns.tolist())

Income Data Shape: (352, 8)

Income Data Columns: ['Household Income Bucket ID', 'Household Income Bucket', 'Year', 'Household Income', 'Household Income Moe', 'Place', 'Place ID', 'share']

Wage Data Shape: (420, 9)

Wage Data Columns: ['Wage Bin ID', 'Wage Bin', 'Year', 'Record Count', 'Total Population', 'Total Population MOE Appx', 'State', 'State ID', 'share']


In [26]:
# Prepare income data - filter for Austin and US
income_austin = income_df[income_df['Place'] == 'Austin, TX'].copy()
income_us = income_df[income_df['Place'] == 'United States'].copy()

# Prepare wage data - filter for Texas and US
wage_texas = wage_df[wage_df['State'] == 'Texas'].copy()
wage_us = wage_df[wage_df['State'] == 'United States'].copy()

# Check available years
print("Income data years:", sorted(income_austin['Year'].unique()))
print("Wage data years:", sorted(wage_texas['Year'].unique()))

Income data years: [np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
Wage data years: [np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


In [27]:
# Create the interactive visualization with subplots
def create_income_wage_comparison(year=2023):
    """
    Create a dual-panel comparison of income and wage distributions
    
    Parameters:
    year: Year to display (default 2023)
    """
    
    # Filter data for the selected year
    income_austin_year = income_austin[income_austin['Year'] == year].copy()
    income_us_year = income_us[income_us['Year'] == year].copy()
    wage_texas_year = wage_texas[wage_texas['Year'] == year].copy()
    wage_us_year = wage_us[wage_us['Year'] == year].copy()
    
    # Sort by income/wage bracket order
    income_austin_year = income_austin_year.sort_values('Household Income Bucket ID')
    income_us_year = income_us_year.sort_values('Household Income Bucket ID')
    wage_texas_year = wage_texas_year.sort_values('Wage Bin ID')
    wage_us_year = wage_us_year.sort_values('Wage Bin ID')
    
    # Create subplots with 1 row and 2 columns
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Household Income Distribution', 'Wage Distribution'),
        horizontal_spacing=0.12
    )
    
    # Define colors
    color_austin_texas = '#FF6B35'  # Warm orange
    color_us = '#4A90B5'  # Cool gray-blue
    
    # Panel 1: Household Income Distribution (Austin vs US)
    fig.add_trace(
        go.Bar(
            x=income_austin_year['Household Income Bucket'],
            y=income_austin_year['share'] * 100,
            name='Austin, TX',
            marker_color=color_austin_texas,
            hovertemplate='<b>%{x}</b><br>' +
                         'Austin: %{y:.2f}%<br>' +
                         '<extra></extra>',
            legendgroup='region1',
            showlegend=True
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=income_us_year['Household Income Bucket'],
            y=income_us_year['share'] * 100,
            name='United States',
            marker_color=color_us,
            hovertemplate='<b>%{x}</b><br>' +
                         'U.S.: %{y:.2f}%<br>' +
                         '<extra></extra>',
            legendgroup='region2',
            showlegend=True
        ),
        row=1, col=1
    )
    
    # Panel 2: Wage Distribution (Texas vs US)
    fig.add_trace(
        go.Bar(
            x=wage_texas_year['Wage Bin'],
            y=wage_texas_year['share'] * 100,
            name='Texas',
            marker_color=color_austin_texas,
            hovertemplate='<b>%{x}</b><br>' +
                         'Texas: %{y:.2f}%<br>' +
                         '<extra></extra>',
            legendgroup='region1',
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(
            x=wage_us_year['Wage Bin'],
            y=wage_us_year['share'] * 100,
            name='United States',
            marker_color=color_us,
            hovertemplate='<b>%{x}</b><br>' +
                         'U.S.: %{y:.2f}%<br>' +
                         '<extra></extra>',
            legendgroup='region2',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Update x-axes
    fig.update_xaxes(title_text="Income Bracket", row=1, col=1, tickangle=-45)
    fig.update_xaxes(title_text="Wage Bracket", row=1, col=2, tickangle=-45)
    
    # Update y-axes
    fig.update_yaxes(title_text="Share of Households (%)", row=1, col=1)
    fig.update_yaxes(title_text="Share of Workers (%)", row=1, col=2)
    
    # Update layout
    fig.update_layout(
        title={
            'text': f'Income and Wage Distribution Comparison ({year})<br>' +
                   '<sub>Comparing Austin/Texas with National Averages</sub>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 20}
        },
        barmode='group',
        height=600,
        width=1600,
        showlegend=True,
        legend=dict(
            title=dict(text='Region'),
            orientation='h',
            yanchor='bottom',
            y=-0.35,
            xanchor='center',
            x=0.5,
            font=dict(size=12)
        ),
        font=dict(size=11),
        hovermode='x unified',
        paper_bgcolor='white',
        plot_bgcolor='rgba(240, 240, 240, 0.5)'
    )
    
    # Add annotations for key insights
    # Annotation 1: Austin's higher $200k+ share
    fig.add_annotation(
        text="Austin's higher $200k+ share<br>reflects wealth concentration",
        xref="x", yref="y",
        x=15, y=income_austin_year[income_austin_year['Household Income Bucket ID'] == 15]['share'].values[0] * 100 + 2,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#FF6B35",
        ax=80,
        ay=-50,
        bordercolor="#FF6B35",
        borderwidth=2,
        borderpad=4,
        bgcolor="rgba(255, 255, 255, 0.9)",
        font=dict(size=10, color="#333333"),
        row=1, col=1
    )
    
    # Annotation 2: Texas wages concentrated in lower brackets
    fig.add_annotation(
        text="Texas wages more concentrated<br>in lower brackets",
        xref="x2", yref="y2",
        x=2, y=wage_texas_year[wage_texas_year['Wage Bin ID'] == 2]['share'].values[0] * 100 + 1,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#FF6B35",
        ax=60,
        ay=-40,
        bordercolor="#FF6B35",
        borderwidth=2,
        borderpad=4,
        bgcolor="rgba(255, 255, 255, 0.9)",
        font=dict(size=10, color="#333333"),
        row=1, col=2
    )
    
    return fig

# Create the visualization for 2023
fig = create_income_wage_comparison(year=2023)
fig.show()

In [28]:
# Create an interactive version with year slider
def create_interactive_comparison_with_slider():
    """
    Create an interactive visualization with a year slider that updates both panels
    """
    
    # Get all available years (common to both datasets)
    years = sorted(list(set(income_austin['Year'].unique()) & set(wage_texas['Year'].unique())))
    
    # Create initial figure for first year
    initial_year = years[0]
    
    # Initialize the figure structure
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Household Income Distribution', 'Wage Distribution'),
        horizontal_spacing=0.12
    )
    
    # Define colors
    color_austin_texas = '#FF6B35'  # Warm orange
    color_us = '#4A90B5'  # Cool gray-blue
    
    # Create frames for animation/slider
    frames = []
    
    for year in years:
        # Filter data for this year
        income_austin_year = income_austin[income_austin['Year'] == year].sort_values('Household Income Bucket ID')
        income_us_year = income_us[income_us['Year'] == year].sort_values('Household Income Bucket ID')
        wage_texas_year = wage_texas[wage_texas['Year'] == year].sort_values('Wage Bin ID')
        wage_us_year = wage_us[wage_us['Year'] == year].sort_values('Wage Bin ID')
        
        frame_data = [
            # Income - Austin
            go.Bar(
                x=income_austin_year['Household Income Bucket'],
                y=income_austin_year['share'] * 100,
                name='Austin, TX',
                marker_color=color_austin_texas,
                hovertemplate='<b>%{x}</b><br>Austin: %{y:.2f}%<br><extra></extra>',
                legendgroup='region1',
                showlegend=True,
                xaxis='x',
                yaxis='y'
            ),
            # Income - US
            go.Bar(
                x=income_us_year['Household Income Bucket'],
                y=income_us_year['share'] * 100,
                name='United States',
                marker_color=color_us,
                hovertemplate='<b>%{x}</b><br>U.S.: %{y:.2f}%<br><extra></extra>',
                legendgroup='region2',
                showlegend=True,
                xaxis='x',
                yaxis='y'
            ),
            # Wage - Texas
            go.Bar(
                x=wage_texas_year['Wage Bin'],
                y=wage_texas_year['share'] * 100,
                name='Texas',
                marker_color=color_austin_texas,
                hovertemplate='<b>%{x}</b><br>Texas: %{y:.2f}%<br><extra></extra>',
                legendgroup='region1',
                showlegend=False,
                xaxis='x2',
                yaxis='y2'
            ),
            # Wage - US
            go.Bar(
                x=wage_us_year['Wage Bin'],
                y=wage_us_year['share'] * 100,
                name='United States',
                marker_color=color_us,
                hovertemplate='<b>%{x}</b><br>U.S.: %{y:.2f}%<br><extra></extra>',
                legendgroup='region2',
                showlegend=False,
                xaxis='x2',
                yaxis='y2'
            )
        ]
        
        frames.append(go.Frame(
            data=frame_data,
            name=str(year),
            layout=go.Layout(
                title_text=f'Income and Wage Distribution Comparison ({year})<br>' +
                          '<sub>Comparing Austin/Texas with National Averages</sub>'
            )
        ))
    
    # Add initial data (first year)
    income_austin_init = income_austin[income_austin['Year'] == initial_year].sort_values('Household Income Bucket ID')
    income_us_init = income_us[income_us['Year'] == initial_year].sort_values('Household Income Bucket ID')
    wage_texas_init = wage_texas[wage_texas['Year'] == initial_year].sort_values('Wage Bin ID')
    wage_us_init = wage_us[wage_us['Year'] == initial_year].sort_values('Wage Bin ID')
    
    fig.add_trace(
        go.Bar(
            x=income_austin_init['Household Income Bucket'],
            y=income_austin_init['share'] * 100,
            name='Austin, TX',
            marker_color=color_austin_texas,
            hovertemplate='<b>%{x}</b><br>Austin: %{y:.2f}%<br><extra></extra>',
            legendgroup='region1'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=income_us_init['Household Income Bucket'],
            y=income_us_init['share'] * 100,
            name='United States',
            marker_color=color_us,
            hovertemplate='<b>%{x}</b><br>U.S.: %{y:.2f}%<br><extra></extra>',
            legendgroup='region2'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=wage_texas_init['Wage Bin'],
            y=wage_texas_init['share'] * 100,
            name='Texas',
            marker_color=color_austin_texas,
            hovertemplate='<b>%{x}</b><br>Texas: %{y:.2f}%<br><extra></extra>',
            legendgroup='region1',
            showlegend=False
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Bar(
            x=wage_us_init['Wage Bin'],
            y=wage_us_init['share'] * 100,
            name='United States',
            marker_color=color_us,
            hovertemplate='<b>%{x}</b><br>U.S.: %{y:.2f}%<br><extra></extra>',
            legendgroup='region2',
            showlegend=False
        ),
        row=1, col=2
    )
    
    # Add frames to figure
    fig.frames = frames
    
    # Update x-axes
    fig.update_xaxes(title_text="Income Bracket", row=1, col=1, tickangle=-45)
    fig.update_xaxes(title_text="Wage Bracket", row=1, col=2, tickangle=-45)
    
    # Update y-axes
    fig.update_yaxes(title_text="Share of Households (%)", row=1, col=1, range=[0, 18])
    fig.update_yaxes(title_text="Share of Workers (%)", row=1, col=2, range=[0, 18])
    
    # Create slider steps
    sliders = [dict(
        active=len(years) - 1,  # Start at most recent year
        yanchor="top",
        y=-0.3,
        xanchor="left",
        x=0.1,
        currentvalue=dict(
            prefix="Year: ",
            visible=True,
            xanchor="center",
            font=dict(size=16)
        ),
        pad=dict(b=10, t=10),
        len=0.8,
        transition=dict(duration=300),
        steps=[
            dict(
                args=[
                    [str(year)],
                    dict(
                        frame=dict(duration=300, redraw=True),
                        mode="immediate",
                        transition=dict(duration=300)
                    )
                ],
                method="animate",
                label=str(year)
            )
            for year in years
        ]
    )]
    
    # Update layout
    fig.update_layout(
        title={
            'text': f'Income and Wage Distribution Comparison ({initial_year})<br>' +
                   '<sub>Comparing Austin/Texas with National Averages</sub>',
            'x': 0.5,
            'xanchor': 'center',
            'font': {'size': 20}
        },
        barmode='group',
        height=650,
        width=1600,
        showlegend=True,
        legend=dict(
            title=dict(text='Region'),
            orientation='h',
            yanchor='bottom',
            y=-0.4,
            xanchor='center',
            x=0.5,
            font=dict(size=12)
        ),
        sliders=sliders,
        font=dict(size=11),
        hovermode='x unified',
        paper_bgcolor='white',
        plot_bgcolor='rgba(240, 240, 240, 0.5)',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                x=0.1,
                y=-0.25,
                xanchor="left",
                yanchor="top",
                pad=dict(r=10, t=10),
                buttons=[
                    dict(
                        label="Play",
                        method="animate",
                        args=[None, dict(
                            frame=dict(duration=500, redraw=True),
                            fromcurrent=True,
                            mode="immediate",
                            transition=dict(duration=300)
                        )]
                    ),
                    dict(
                        label="Pause",
                        method="animate",
                        args=[[None], dict(
                            frame=dict(duration=0, redraw=False),
                            mode="immediate",
                            transition=dict(duration=0)
                        )]
                    )
                ]
            )
        ]
    )
    
    # Add annotations for key insights (will be visible for initial year)
    # These annotations highlight the key story
    last_year_data = years[-1]
    income_austin_last = income_austin[income_austin['Year'] == last_year_data].sort_values('Household Income Bucket ID')
    wage_texas_last = wage_texas[wage_texas['Year'] == last_year_data].sort_values('Wage Bin ID')
    
    fig.add_annotation(
        text="<b>Austin's higher $200k+ share<br>reflects wealth concentration</b>",
        xref="x", yref="y",
        x=15, y=income_austin_last[income_austin_last['Household Income Bucket ID'] == 15]['share'].values[0] * 100 + 1.5,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#FF6B35",
        ax=80,
        ay=-50,
        bordercolor="#FF6B35",
        borderwidth=2,
        borderpad=4,
        bgcolor="rgba(255, 255, 255, 0.95)",
        font=dict(size=10, color="#333333"),
        row=1, col=1
    )
    
    fig.add_annotation(
        text="<b>Texas wages more concentrated<br>in lower brackets</b>",
        xref="x2", yref="y2",
        x=2, y=wage_texas_last[wage_texas_last['Wage Bin ID'] == 2]['share'].values[0] * 100 + 1,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        arrowcolor="#FF6B35",
        ax=60,
        ay=-40,
        bordercolor="#FF6B35",
        borderwidth=2,
        borderpad=4,
        bgcolor="rgba(255, 255, 255, 0.95)",
        font=dict(size=10, color="#333333"),
        row=1, col=2
    )
    
    return fig

# Create the interactive visualization with slider
fig_interactive = create_interactive_comparison_with_slider()
fig_interactive.show()